In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [ ]:
def standardize_county_name(county):
    """
    Standardize Washington county names to handle all variations
    """
    if pd.isna(county):
        return 'Unknown'

    county = str(county).upper().strip()

    # Remove common suffixes and prefixes
    county = county.replace(' COUNTY', '')
    county = county.replace('CITY OF ', '')

    # Dictionary of corrections for Washington counties
    corrections = {
        # Case variations
        'Clark': 'CLARK',
        'CLALLAM': 'CLALLAM',
        'Walla Walla': 'WALLA WALLA',
        'King': 'KING',
        'KING': 'KING',
        'Kings': 'KING',
        'THURSTON': 'THURSTON',
        'Thurston': 'THURSTON',
        'SPOKANE': 'SPOKANE',
        'Spokane': 'SPOKANE',

        # Spacing and formatting
        'GRAYS HARBOR': 'GRAYS HARBOR',
        'GRAYSHARBOR': 'GRAYS HARBOR',
        'Grays Harbor': 'GRAYS HARBOR',

        # Common misspellings
        'WAHKIAKUM': 'WAHKIAKUM',
        'WAHKIAKIMI': 'WAHKIAKUM',
        'Wahkiakum': 'WAHKIAKUM',

        # Remove districts
        'District of Columbia': 'COLUMBIA',
        'PRINCE GEORGE\'S': 'PRINCE GEORGE',

        # Standardize variations
        'ASOTIN': 'ASOTIN',
        'Asotin': 'ASOTIN',
        'BENTON': 'BENTON',
        'Benton': 'BENTON',
        'CHELAN': 'CHELAN',
        'Chelan': 'CHELAN',
        'CLACKAMAS': 'CLACKAMAS',
        'COWLITZ': 'COWLITZ',
        'Cowlitz': 'COWLITZ',
        'DOUGLAS': 'DOUGLAS',
        'Douglas': 'DOUGLAS',
        'FERRY': 'FERRY',
        'Ferry': 'FERRY',
        'FRANKLIN': 'FRANKLIN',
        'Franklin': 'FRANKLIN',
        'GRANT': 'GRANT',
        'Grant': 'GRANT',
        'ISLAND': 'ISLAND',
        'Island': 'ISLAND',
        'JEFFERSON': 'JEFFERSON',
        'Jefferson': 'JEFFERSON',
        'KITSAP': 'KITSAP',
        'Kitsap': 'KITSAP',
        'KITTITAS': 'KITTITAS',
        'Kittitas': 'KITTITAS',
        'KLICKITAT': 'KLICKITAT',
        'Klickitat': 'KLICKITAT',
        'LEWIS': 'LEWIS',
        'Lewis': 'LEWIS',
        'LINCOLN': 'LINCOLN',
        'Lincoln': 'LINCOLN',
        'MASON': 'MASON',
        'Mason': 'MASON',
        'OKANOGAN': 'OKANOGAN',
        'Okanogan': 'OKANOGAN',
        'PACIFIC': 'PACIFIC',
        'Pacific': 'PACIFIC',
        'PEND OREILLE': 'PEND OREILLE',
        'Pend Oreille': 'PEND OREILLE',
        'PIERCE': 'PIERCE',
        'Pierce': 'PIERCE',
        'SAN JUAN': 'SAN JUAN',
        'San Juan': 'SAN JUAN',
        'SKAGIT': 'SKAGIT',
        'Skagit': 'SKAGIT',
        'SKAMANIA': 'SKAMANIA',
        'Skamania': 'SKAMANIA',
        'SNOHOMISH': 'SNOHOMISH',
        'Snohomish': 'SNOHOMISH',
        'STEVENS': 'STEVENS',
        'Stevens': 'STEVENS',
        'WHATCOM': 'WHATCOM',
        'Whatcom': 'WHATCOM',
        'WHITMAN': 'WHITMAN',
        'Whitman': 'WHITMAN',
        'YAKIMA': 'YAKIMA',
        'Yakima': 'YAKIMA',

        # Remove non-Washington counties
        'New York': 'OTHER',
        'Los Angeles': 'OTHER',
        'Sacramento': 'OTHER',
        'Alameda': 'OTHER',
        'San Mateo': 'OTHER',
        'Fairfax (city)': 'OTHER',
        'Blue Earth': 'OTHER',
        'Suffolk': 'OTHER',
        'Arlington': 'OTHER',
        'Montgomery': 'OTHER',
        'Hennepin': 'OTHER',
        'Dallas': 'OTHER',
        'Harris': 'OTHER',
        'Hudson': 'OTHER',
        'Lake': 'OTHER',
        'St. Louis': 'OTHER',
        'Marin': 'OTHER',
        'Various': 'OTHER'
    }

    return corrections.get(county, county)

In [ ]:
def clean_campaign_finance_data(df):
    """
    Clean and preprocess Washington state campaign finance data
    """
    df_clean = df.copy()

    # 1. Handle Missing Values
    numeric_columns = [
        'contributions_amount', 'carryforward_amount', 'expenditures_amount',
        'loans_amount', 'pledges_amount', 'debts_amount',
        'independent_expenditures_for_amount', 'independent_expenditures_against_amount'
    ]
    df_clean[numeric_columns] = df_clean[numeric_columns].fillna(0)

    categorical_columns = [
        'filer_type', 'filing_type', 'candidate_committee_status',
        'committee_category', 'political_committee_type', 'party',
        'election_status', 'ballot_committee'
    ]
    df_clean[categorical_columns] = df_clean[categorical_columns].fillna('Unknown')

    # 2. Date Processing
    date_columns = ['registered', 'declared', 'withdrew', 'discontinued', 'receipt_date', 'election_date']
    for col in date_columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

    # 3. Standardize County Names
    df_clean['committee_county'] = df_clean['committee_county'].apply(standardize_county_name)

    # 4. Define Washington Regions
    wa_regions = {
        'Puget Sound': ['KING', 'PIERCE', 'SNOHOMISH', 'KITSAP'],
        'Western WA': ['CLARK', 'COWLITZ', 'GRAYS HARBOR', 'LEWIS',
                      'PACIFIC', 'WAHKIAKUM', 'THURSTON', 'MASON'],
        'Eastern WA': ['SPOKANE', 'WHITMAN', 'ADAMS', 'LINCOLN',
                      'FERRY', 'STEVENS', 'PEND OREILLE'],
        'Central WA': ['YAKIMA', 'BENTON', 'FRANKLIN', 'GRANT',
                      'DOUGLAS', 'CHELAN', 'OKANOGAN', 'KITTITAS'],
        'Coastal': ['CLALLAM', 'JEFFERSON', 'PACIFIC', 'GRAYS HARBOR',
                   'WHATCOM', 'SKAGIT', 'SAN JUAN', 'ISLAND']
    }

    def get_wa_region(county):
        for region, counties in wa_regions.items():
            if county in counties:
                return region
        return 'Other'

    df_clean['region'] = df_clean['committee_county'].apply(get_wa_region)

    # 5. Create Derived Fields
    df_clean['total_financial_activity'] = (
        df_clean['contributions_amount'] +
        df_clean['expenditures_amount'] +
        df_clean['loans_amount']
    )

    df_clean['contribution_expenditure_ratio'] = np.where(
        df_clean['expenditures_amount'] > 0,
        df_clean['contributions_amount'] / df_clean['expenditures_amount'],
        0
    )

    df_clean['is_active'] = np.where(
        (df_clean['withdrew'].isna()) &
        (df_clean['discontinued'].isna()) &
        (df_clean['active_candidate'] == True),
        1, 0
    )

    # 6. Data Validation
    for col in numeric_columns:
        df_clean = df_clean[df_clean[col] >= 0]

    # 7. Time-based Features
    # df_clean['year'] = df_clean['receipt_date'].dt.year
    # df_clean['month'] = df_clean['receipt_date'].dt.month
    # df_clean['quarter'] = df_clean['receipt_date'].dt.quarter

    # 8. Contribution Categories
    df_clean['contribution_category'] = pd.cut(
        df_clean['contributions_amount'],
        bins=[0, 1000, 5000, 10000, 50000, float('inf')],
        labels=['Small', 'Medium', 'Large', 'Very Large', 'Mega']
    )

    # # 9. Data Type Optimization
    # boolean_columns = ['active_candidate', ]
    # for col in boolean_columns:
    #     df_clean[col] = df_clean[col].map({'Y': True, 'N': False})

    # df_clean[categorical_columns] = df_clean[categorical_columns].astype('category')

    return df_clean

In [ ]:
data_path = '/content/drive/MyDrive/VAD_Final_Project/'
# Read the data
df_new = pd.read_csv(data_path+'campaign_finance_data.csv')

<ipython-input-111-220dc2127d9b>:3: DtypeWarning: Columns (3,4,5,6,10,12,19,21,28,29,32,33,34,35,39,40,41,42,43,44,45,46,48,49,50,51,52,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(data_path+'campaign_finance_data.csv')


In [ ]:
df_new_clean1 = df_new[~df_new.contributions_amount.isnull()]

In [ ]:
df_new_clean1_copy = df_new_clean1.copy()

In [ ]:
df_new_clean2_main = df_new_clean1_copy[~df_new_clean1_copy['committee_county'].isnull()]
# df_new_clean2_main = df_new_clean2_main[df_new_clean2_main['active_candidate'] == True]

In [ ]:
cleaned_data = clean_campaign_finance_data(df_new_clean2_main)

<ipython-input-110-5abe6fb76746>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-110-5abe6fb76746>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-110-5abe6fb76746>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-110-5abe6fb76746>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateut

In [ ]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10148 entries, 0 to 52265
Data columns (total 81 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       10148 non-null  object        
 1   filer_id                                 10148 non-null  object        
 2   filer_type                               10148 non-null  object        
 3   registered                               9801 non-null   datetime64[ns]
 4   declared                                 3403 non-null   datetime64[ns]
 5   withdrew                                 5 non-null      datetime64[ns]
 6   discontinued                             20 non-null     datetime64[ns]
 7   filing_type                              10148 non-null  object        
 8   receipt_date                             10147 non-null  datetime64[ns]
 9   election_year                            101

In [ ]:
# Define the 10 election periods (adjust the years as necessary)
# election_periods = [2012, 2014, 2016, 2018, 2020, 2022, 2024]

# Filter the dataset for the specified election periods
# cleaned_data_2 = cleaned_data[cleaned_data['election_year'].isin(election_periods)]

In [ ]:
columns_to_keep = [
    'id',
    'filer_name',
    'filer_type',
    'filing_type',
    'receipt_date',
    'election_year',
    'contributions_amount',
    'expenditures_amount',
    'total_financial_activity',
    'contribution_expenditure_ratio',
    'committee_category',
    'contribution_category',
    'committee_state',
    'committee_county',
    'party',
    'region',
    'is_active'
]

# Filter the dataset to keep only these columns
cleaned_data_2 = cleaned_data[columns_to_keep]
cleaned_data_2 = cleaned_data_2[~(cleaned_data_2.receipt_date == '1899-12-31')]
cleaned_data_2 = cleaned_data_2[~cleaned_data_2.contribution_category.isnull()]

# Save the cleaned dataset to an Excel file
# cleaned_file_path = 'cleaned_campaign_finance_data_reduced_updated.xlsx'
# cleaned_data_2.to_excel(cleaned_file_path, index=False)

In [ ]:
# cleaned_file_path2 = 'cleaned_campaign_finance_data_reduced.csv'
# cleaned_data_2.to_csv(cleaned_file_path2, index=False)

In [ ]:
cleaned_data_2.head(5)

,id,filer_name,filer_type,filing_type,receipt_date,election_year,contributions_amount,expenditures_amount,total_financial_activity,contribution_expenditure_ratio,committee_category,contribution_category,committee_state,committee_county,party,region,is_active
0,ca-2024-93123,Kevin W. Van De Wege,CA,Electronic,2021-01-19,2024,32202.89,37509.32,69712.21,0.858530,Candidate,Very Large,WA,CLALLAM,DEMOCRATIC,Coastal,1
18,ca-2024-689015,My-Linh Thai,CA,Electronic,2023-01-02,2024,86624.46,76428.15,163052.61,1.133410,Candidate,Mega,WA,KING,Unknown,Puget Sound,1
19,ca-2024-3296585,Wayne Fournier,CA,Electronic,2024-02-12,2024,18006.00,9411.91,27417.91,1.913108,Candidate,Very Large,WA,THURSTON,DEMOCRATIC,Western WA,1
33,co-2024-425,ALLIANCE FOR A COMPETITIVE ECONOMY OF GREATER ...,CO,Electronic,2009-06-01,2024,15500.00,12916.54,28416.54,1.200012,Other,Very Large,WA,VARIOUS,Unknown,Other,0
34,ca-2024-3310897,Rita Cole Hall,CA,Electronic,2024-06-28,2024,1395.99,1002.35,2654.33,1.392717,Candidate,Medium,WA,PACIFIC,REPUBLICAN,Western WA,1


In [ ]:
cleaned_data_2.committee_state.unique()

array(['WA', 'Washington', 'Wa', 'wa', 'United States (+1)', 'WASHINGTON',
       'WASHINGTON (WA)', nan, 'WA.', 'Arizona', 'USA', 'WA - WASHINGTON',
       'United States (1)', 'King', 'WAStates (1)', 'washington',
       'Waahington', 'Wa.', 'Washington state', 'Walla', 'United States',
       'Franklin', 'WASINGTON', 'Washington(WA)', 'WA - Washington', 'CA',
       'US', 'Washinton'], dtype=object)

In [ ]:
cleaned_data_2.contributions_amount.unique()

array([32202.89, 86624.46, 18006.  , ...,  1993.49,  5188.37,  5565.89])

In [ ]:
cleaned_data_2[cleaned_data_2.receipt_date.isnull()]

(1173, 17)

In [ ]:
cleaned_data_2.election_year.max()

2029

In [ ]:
cleaned_data_2.shape

(10147, 17)

In [ ]:
cleaned_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8974 entries, 0 to 52265
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              8974 non-null   object        
 1   filer_name                      8974 non-null   object        
 2   filer_type                      8974 non-null   object        
 3   filing_type                     8974 non-null   object        
 4   receipt_date                    8973 non-null   datetime64[ns]
 5   election_year                   8974 non-null   int64         
 6   contributions_amount            8974 non-null   float64       
 7   expenditures_amount             8974 non-null   float64       
 8   total_financial_activity        8974 non-null   float64       
 9   contribution_expenditure_ratio  8974 non-null   float64       
 10  committee_category              8974 non-null   object        
 11  contribu

In [ ]:
df_new_clean1_copy.active_candidate.value_counts()

,count
active_candidate,
True,10962
False,123


In [ ]:
cleaned_data = clean_campaign_finance_data(df_new)

<ipython-input-13-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-13-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-13-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-13-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`.

In [ ]:
cleaned_data[cleaned_data.committ.isnull()]

,id,filer_id,filer_type,registered,declared,withdrew,discontinued,filing_type,receipt_date,election_year,...,continuing,updated_at,region,total_financial_activity,contribution_expenditure_ratio,is_active,year,month,quarter,contribution_category


In [ ]:
cleaned_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51203 entries, 0 to 52265
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              51203 non-null  object        
 1   filer_name                      51203 non-null  object        
 2   receipt_date                    43784 non-null  datetime64[ns]
 3   election_year                   51203 non-null  int64         
 4   contributions_amount            51203 non-null  float64       
 5   expenditures_amount             51203 non-null  float64       
 6   total_financial_activity        51203 non-null  float64       
 7   contribution_expenditure_ratio  51203 non-null  float64       
 8   committee_category              51203 non-null  category      
 9   contribution_category           18167 non-null  category      
 10  committee_state                 30251 non-null  object        
 11  committ

In [ ]:
df_camp_clean[df_camp_clean['committee_id'] == 12199][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email
27767,NaN,22056 Se 304Th St,Black Diamond,King,NaN,98010,RJMAURO99@MSN.COM,rjmauro99@msn.com


In [ ]:
df_camp_clean.columns

Index(['id', 'filer_id', 'filer_type', 'registered', 'declared', 'withdrew',
       'discontinued', 'filing_type', 'receipt_date', 'election_year',
       'candidate_committee_status', 'filer_name', 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email', 'candidate_committee_phone', 'office', 'office_code',
       'jurisdiction', 'jurisdiction_code', 'jurisdiction_county',
       'jurisdiction_type', 'jurisdiction_voters',
       'jurisdiction_reporting_code', 'jurisdiction_reporting_requirement',
       'committee_category', 'political_committee_type', 'bonafide_committee',
       'bonafide_type', 'position', 'party_code', 'party', 'election_date',
       'reporting_option', 'active_candidate', 'on_primary_election_ballot',
       'on_general_election_ballot', 'primary_election_status',
       'general_election_status', 'election_status', 'exempt_nonexempt',
       'b

In [ ]:
df_new.committee_county.unique()

array(['CLALLAM', nan, 'KING', 'THURSTON', 'Various', 'PACIFIC', 'KITSAP',
       'COWLITZ', 'BENTON', 'WHITMAN', 'SNOHOMISH', 'YAKIMA', 'WHATCOM',
       'CHELAN', 'STEVENS', 'MASON', 'PIERCE', 'ASOTIN', 'CLARK',
       'OKANOGAN', 'LEWIS', 'KITTITAS', 'GRANT', 'SPOKANE', 'ISLAND',
       'ADAMS', 'SKAMANIA', 'WALLA WALLA', 'FERRY', 'JEFFERSON', 'SKAGIT',
       'GRAYS HARBOR', 'KLICKITAT', 'WAHKIAKUM', 'SAN JUAN', 'LINCOLN',
       'PEND OREILLE', 'FRANKLIN', 'DOUGLAS', 'COLUMBIA', 'GARFIELD'],
      dtype=object)

In [ ]:
df_new.committee_state.unique()

array(['WA', nan, 'DC', 'Washington', 'Wa', 'C3 State Report', 'OR', 'wa',
       'w', 'MD', 'NY', 'MN', 'WASHINGTON', 'VA', 'United States',
       'WA - Washington', 'United States (+1)', 'W.a', 'CA', 'TX', 'NE',
       'TN', 'US', 'WASHINGTON (WA)', 'ID', 'KS', 'NJ', 'OH', 'VT', 'IL',
       'MO', 'WA.', '56', 'washington', '68', 'WY', 'Arizona',
       'WA - WASHINGTON', 'USA', 'United States (1)', 'King', 'WA 98145',
       'Select A County', 'Washington (WA)', 'Wa.', 'WAStates (1)',
       'Washingon', 'SC', 'Waahington', 'id_495', 'Grant', 'Home',
       'Washington state', 'Walla', '4426', 'Clark', '48', '47',
       'Washihgton', 'WA Washington', 'Franklin', 'WASINGTON', 'wa.',
       'Stevens', 'Wash', 'Skamania', 'Washinton', 'Washington State',
       'Washinhton', 'US-WA', 'Washington(WA)', 'WA`', 'WAq', 'W',
       'string:WA', '700', 'Washing', 'Snohomish', '3462',
       '9227f6f4-cf63-e411-8d8b-005056ba7b6d', 'KING', 'Lewis',
       '-- select state --', 'WaWashington'

In [ ]:
df_new[df_new['committee_state']=='Select A County'][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
20658,NaN,3004 island view ct,gig harbor,PIERCE,Select A County,98335,abersolds@psd401.net,abersolds@psd401.net,NaN
49153,NaN,402 N CHESTNUT ST,ELLENSBURG,KITTITAS,Select A County,98926,ludlummeg@gmail.com,ludlummeg@gmail.com,NaN


In [ ]:
def create_dashboard_views(df):
    """
    Create comprehensive views for all four dashboards
    """
    views = {}

    # OVERVIEW DASHBOARD VIEWS
    views['overview_kpis'] = df.groupby(['year', 'month']).agg({
        'contributions_amount': 'sum',
        'expenditures_amount': 'sum',
        'committee_id': 'nunique',
        'is_active': 'sum'
    }).reset_index()

    views['overview_trends'] = df.groupby(['year', 'month', 'party']).agg({
        'contributions_amount': 'sum',
        'expenditures_amount': 'sum',
        'committee_id': 'count'
    }).reset_index()

    # CONTRIBUTIONS DASHBOARD VIEWS
    views['contribution_analysis'] = df.groupby(['committee_county', 'year', 'month']).agg({
        'contributions_amount': 'sum',
        'contribution_category': lambda x: x.value_counts().to_dict(),
        'committee_id': 'nunique'
    }).reset_index()

    views['party_contributions'] = df.groupby(['party', 'year', 'month']).agg({
        'contributions_amount': 'sum',
        'committee_id': 'nunique'
    }).reset_index()

    # GEOGRAPHIC DASHBOARD VIEWS
    views['geographic_summary'] = df.groupby(['committee_county', 'region']).agg({
        'contributions_amount': 'sum',
        'expenditures_amount': 'sum',
        'committee_id': 'nunique',
        'is_active': 'sum',
        'party': lambda x: x.value_counts().to_dict()
    }).reset_index()

    views['county_trends'] = df.groupby(['committee_county', 'year', 'month']).agg({
        'contributions_amount': 'sum',
        'expenditures_amount': 'sum'
    }).reset_index()

    # COMMITTEE DASHBOARD VIEWS
    views['committee_performance'] = df.groupby(['committee_id', 'committee_category', 'party']).agg({
        'contributions_amount': 'sum',
        'expenditures_amount': 'sum',
        'is_active': 'max',
        'committee_county': 'first'
    }).reset_index()

    views['committee_trends'] = df.groupby(['committee_category', 'year', 'month']).agg({
        'contributions_amount': 'sum',
        'committee_id': 'nunique'
    }).reset_index()

    return views

In [ ]:
df_new[df_new['committee_state']=='-- select state --'][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
51611,NaN,11670 River Bend Dr,Leavenworth,CHELAN,-- select state --,98826,goehner@live.com,goehner@live.com,NaN


In [ ]:
df_new[df_new['committee_state']=='4426'][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
42217,NaN,18903 SE 44th Ct,Issaquah,KING,4426,98027,tim.flood@hotmail.com,tim.flood@hotmail.com,NaN


In [ ]:
df_new[df_new['committee_state']=='C3 STATE REPORT'][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount


In [ ]:
df_new[df_new['committee_id'] == 1][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
4582,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,5548232.39
4589,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,1301.71
4591,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,NaN
4595,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,NaN
4597,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,NaN
4602,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,NaN
4605,NaN,PO BOX 4187,SEATTLE,NaN,WA,98194,COMPLIANCE@BLUEWAVEPOLITICS.COM,NaN,NaN


In [ ]:
df_new[df_new.duplicated(subset=['committee_id'])][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
2,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,32805.76
4,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,0.00
7,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
8,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
9,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35999,NaN,NaN,NaN,SPOKANE,NaN,NaN,NaN,NaN,NaN
36121,NaN,NaN,NaN,YAKIMA,NaN,NaN,NaN,NaN,NaN
36147,NaN,NaN,NaN,STEVENS,NaN,NaN,NaN,NaN,NaN
48145,NaN,7621 Steamboat Is Rd NW,Olympia,THURSTON,Wa,98502,osterbergjf@gmail.com,willameen@comcast.net,NaN


In [ ]:
df_new.shape

(52266, 76)

In [ ]:
df_new[df_new.contributions_amount.isnull()][[ 'committee_acronym',
       'committee_address', 'committee_city', 'committee_county',
       'committee_state', 'committee_zip', 'committee_email',
       'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
3,NaN,6710 188th St SE,Snohomish,NaN,WA,98296,Suemagoo3802@yahoo.com,NaN,NaN
5,NaN,4444 44TH AVE SW #405,SEATTLE,NaN,NaN,98116,NaN,NaN,NaN
6,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
7,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
8,NaN,PO BOX 2595,YELM,NaN,WA,98597,SMKMMBERG@MSN.COM,NaN,NaN
...,...,...,...,...,...,...,...,...,...
52250,NaN,PO Box 74,Curlew,FERRY,WA,99118,teresa@vulcanmountain.com,teresa@vulcanmountain.com,NaN
52254,DCT,General delivery,Olympia,NaN,Wa,98502,toliverforgovernor@proton.me,desireetoliverforoly@gmail.com,NaN
52255,NaN,PO Box 12066,Seattle,KING,WA,98102,wise.julie@me.com,wise.julie@me.com,NaN
52262,NaN,17806 111TH STREET CT E,BONNEY LAKE,PIERCE,WA,98391,JUSTINEVANS78@GMAIL.COM,NaN,NaN


In [ ]:
df_camp_clean.shape

(51203, 84)

In [ ]:
 df_new_clean1[(df_new_clean1.committee_county.isnull()) and (~df_new_clean1.committee_zip.isnull())][[ 'committee_acronym',
                                                         'committee_address', 'committee_city', 'committee_county',
                                                          'committee_state', 'committee_zip', 'committee_email',
                                                          'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
1,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,981.62
2,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,32805.76
4,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,0.00
13,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,10130.00
14,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,NaN,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,385.00
...,...,...,...,...,...,...,...,...,...
51507,NaN,PO Box 22169,Seattle,NaN,WA,98122,info@electbobferguson.com,info@electbobferguson.com,1550213.12
51580,NaN,108 south State street apt 109,Spokane,NaN,Washington,99201-5130,votekriss@gmail.com,votekriss@gmail.com,438.11
51592,NaN,P.O. Box 6182,Kennewick,NaN,WA,99336,info@klippertforospi.com,info@klippertforospi.com,8385.00
52198,NaN,PO Box 61856,Vancouver,NaN,WA,98666-1856,bernardveljacic@gmail.com,bernardveljacic@gmail.com,0.00


In [ ]:
# df_new_clean2 = df_new_clean1[(~df_new_clean1.committee_zip.isnull())]

In [ ]:
#  df_new_clean1[(df_new_clean1.committee_county.isnull()) & (~df_new_clean1.committee_zip.isnull())][[ 'committee_acronym',
#                                                          'committee_address', 'committee_city', 'committee_county',
#                                                           'committee_state', 'committee_zip', 'committee_email',
#                                                           'candidate_email','contributions_amount']] (df_new_clean1.committee_county.isnull()) &

In [ ]:
print(df_new_clean1.shape)
# print(df_new_clean2.shape)

(22155, 76)


In [ ]:
df_new_clean2.committee_zip.unique()

array(['98382', '98684-8428', '99362', ..., '98666-1856', '98010-9799',
       '98530'], dtype=object)

In [ ]:
# # !pip install pgeocode
# import pgeocode
# import pandas as pd

# # Load your DataFrame (df_new_clean2_interim)
# # Example for creating dummy data
# # df_new_clean2_interim = pd.DataFrame({'committee_zip': ['98684-8428', '99362', '98168'], 'committee_county': [None, None, None]})

# # Initialize the nominatim object for US ZIP codes
# nomi = pgeocode.Nominatim('US')

# # Function to get the county name
# def get_county(zip_code):
#     base_zip = zip_code.split('-')[0]  # Handle ZIP+4 format
#     location = nomi.query_postal_code(base_zip)
#     if location is not None:
#         return location.county_name
#     else:
#         return None

# # Update the 'committee_county' column
# df_new_clean1_copy['committee_county'] = df_new_clean1_copy.apply(
#     lambda row: get_county(row['committee_zip']) if pd.isnull(row['committee_county']) else row['committee_county'],
#     axis=1
# )

# # Display the updated DataFrame
# print(df_new_clean2_interim)

                    id    filer_id filer_type              registered  \
0        ca-2024-93123  VANDK  382         CA  01/19/2021 12:00:00 AM   
1            co-2011-8  17THDR 683         CO                     NaN   
2            co-2012-8  17THDR 683         CO                     NaN   
4            co-2013-8  17THDR 683         CO                     NaN   
13           co-2014-8  17THDR 683         CO                     NaN   
...                ...         ...        ...                     ...   
52259   ca-2026-689072  TORRN--301         CA  01/12/2023 12:00:00 AM   
52260  ca-2027-3279394  FELLF  109         CA  01/18/2024 12:00:00 AM   
52261  ca-2024-3311816  PEREE--957         CA  05/14/2024 12:00:00 AM   
52263  ca-2024-3296715  HOFEF  639         CA  03/28/2024 12:00:00 AM   
52265  ca-2024-3296780  MELMJ--888         CA  04/12/2024 12:00:00 AM   

                     declared withdrew discontinued filing_type receipt_date  \
0                         NaN      NaN     

In [ ]:
import pgeocode
import pandas as pd

# Initialize the nominatim object for US ZIP codes
nomi = pgeocode.Nominatim('US')

# Function to get the county name
def get_county(zip_code):
    try:
        # Handle ZIP+4 format by splitting and using base ZIP
        base_zip = str(zip_code).split('-')[0]
        location = nomi.query_postal_code(base_zip)
        if location is not None and pd.notnull(location.county_name):
            return location.county_name
        else:
            return None
    except Exception as e:
        return None

# Apply the function to the unmapped rows
df_new_clean1_copy['committee_county'] = df_new_clean1_copy.apply(
    lambda row: get_county(row['committee_zip']) if pd.isnull(row['committee_county']) else row['committee_county'],
    axis=1
)

# Check rows that still have missing counties
unmapped_zips = df_new_clean1_copy[df_new_clean1_copy['committee_county'].isnull()]

# Log unmapped ZIP codes for manual inspection
print("Unmapped ZIP codes:")
print(unmapped_zips['committee_zip'].unique())

# Optionally, save the remaining unmapped entries to a CSV for manual review
unmapped_zips.to_csv('unmapped_zips.csv', index=False)

# Save the updated DataFrame to a CSV
df_new_clean1_copy.to_csv('updated_committee_data_with_all_counties.csv', index=False)

# Print the updated DataFrame
print("Updated DataFrame with county mapping:")
df_new_clean1_copy


Unmapped ZIP codes:
['980933522' '980023384' '980312091' '984990556' '980645882' '982610296'
 '980347501' '98697' '981991011' '992233013' '986470321' '980460365'
 '988071824' '988020212' '981390513' '983601015' '982751182' '981113854'
 '993501189' '983330243' '980121391' '982720764' '38102' '9206-3601'
 '980579798' '980091153' '992239998' '207722687' '72883' '983921024'
 '985128108' '984970164' '980720981' '981445407' '992040086' '992248223'
 '98205' '980209516' '981212411' '983722000' '985700151' '980401192'
 '985070658' '985076328' '983870875' '986728908' '983706233' '986872382'
 '981341914' '980365719' '981130149' '980090728' '981056191' '980830817'
 '985699371' '984075810' '981554002' '989263325' '982502431' '981113481'
 '985077704' '984112393' '981011300' '981334419' '980710124' '981102648'
 '980923700' '993527601' '993620023' '981341823' '985093131' '982062612'
 '989072129' '983370204' '992141716' '980588989' '981034128' '982214183'
 '982210653' '982293816' '982270812' '984011743

,id,filer_id,filer_type,registered,declared,withdrew,discontinued,filing_type,receipt_date,election_year,...,url,committee_id,person_id,candidacy_id,fund_id,legislative_district,treasurer_name,pac_type,continuing,updated_at
0,ca-2024-93123,VANDK 382,CA,01/19/2021 12:00:00 AM,NaN,NaN,NaN,Electronic,01/19/2021,2024,...,https://apollo.pdc.wa.gov/public/registrations...,26492.0,26330.0,93123.0,17290.0,24.0,JASON BENNETT,candidate,NaN,08/26/2024 07:09:08 PM
1,co-2011-8,17THDR 683,CO,NaN,NaN,NaN,NaN,Electronic,03/30/2008,2011,...,https://web.pdc.wa.gov/rptimg/default.aspx?doc...,8.0,32738.0,NaN,10871.0,NaN,LARRY DEVRIES,bonafide,True,09/19/2024 10:32:09 AM
2,co-2012-8,17THDR 683,CO,NaN,NaN,NaN,NaN,Electronic,03/30/2008,2012,...,https://web.pdc.wa.gov/rptimg/default.aspx?doc...,8.0,32738.0,NaN,9631.0,NaN,LARRY DEVRIES,bonafide,True,09/19/2024 10:32:09 AM
4,co-2013-8,17THDR 683,CO,NaN,NaN,NaN,NaN,Electronic,03/30/2008,2013,...,https://web.pdc.wa.gov/rptimg/default.aspx?doc...,8.0,32738.0,NaN,8641.0,NaN,LARRY DEVRIES,bonafide,True,09/19/2024 10:32:09 AM
13,co-2014-8,17THDR 683,CO,NaN,NaN,NaN,NaN,Electronic,03/30/2008,2014,...,https://web.pdc.wa.gov/rptimg/default.aspx?doc...,8.0,32738.0,NaN,7430.0,NaN,LARRY DEVRIES,bonafide,True,09/19/2024 10:32:09 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52259,ca-2026-689072,TORRN--301,CA,01/12/2023 12:00:00 AM,NaN,NaN,NaN,Electronic,01/12/2023,2026,...,https://apollo.pdc.wa.gov/public/registrations...,31619.0,31599.0,689072.0,19881.0,16.0,Jason Michaud,candidate,NaN,09/04/2024 07:45:14 PM
52260,ca-2027-3279394,FELLF 109,CA,01/18/2024 12:00:00 AM,NaN,NaN,NaN,Electronic,01/18/2024,2027,...,https://apollo.pdc.wa.gov/public/registrations...,35502.0,1180.0,3279394.0,23762.0,NaN,Andy Lo,candidate,NaN,09/07/2024 08:03:56 AM
52261,ca-2024-3311816,PEREE--957,CA,05/14/2024 12:00:00 AM,05/10/2024 12:00:00 AM,NaN,NaN,Electronic,05/14/2024,2024,...,https://apollo.pdc.wa.gov/public/registrations...,36561.0,46193.0,3311816.0,24360.0,14.0,Eddie Perez,candidate,NaN,08/29/2024 07:03:22 AM
52263,ca-2024-3296715,HOFEF 639,CA,03/28/2024 12:00:00 AM,05/09/2024 12:00:00 AM,NaN,NaN,Electronic,03/28/2024,2024,...,https://apollo.pdc.wa.gov/public/registrations...,36100.0,2816.0,3296715.0,24288.0,NaN,Krystal Patton,candidate,NaN,08/29/2024 07:03:25 AM


In [ ]:
import pgeocode
import pandas as pd

# Initialize the nominatim object for US ZIP codes
nomi = pgeocode.Nominatim('US')

# Function to format ZIP codes into ZIP+4 if necessary
def format_zip(zip_code):
    zip_str = str(zip_code)
    if len(zip_str) == 9 and zip_str.isdigit():  # If it's 9 digits and numeric
        return zip_str[:5] + '-' + zip_str[5:]
    elif len(zip_str) == 5 and zip_str.isdigit():  # If it's 5 digits
        return zip_str
    else:
        return zip_str  # Leave other formats unchanged

# Function to get the county name
def get_county(zip_code):
    try:
        formatted_zip = format_zip(zip_code)  # Ensure proper formatting
        base_zip = formatted_zip.split('-')[0]  # Use only the first 5 digits for lookup
        location = nomi.query_postal_code(base_zip)
        if location is not None and pd.notnull(location.county_name):
            return location.county_name
        else:
            return None
    except Exception as e:
        return None

# Apply formatting to ZIP codes and update the county mapping
df_new_clean1_copy['committee_zip'] = df_new_clean1_copy['committee_zip'].apply(format_zip)

# Update the 'committee_county' column using the formatted ZIP codes
df_new_clean1_copy['committee_county'] = df_new_clean1_copy.apply(
    lambda row: get_county(row['committee_zip']) if pd.isnull(row['committee_county']) else row['committee_county'],
    axis=1
)

# Check rows that still have missing counties
unmapped_zips = df_new_clean1_copy[df_new_clean1_copy['committee_county'].isnull()]

# Log unmapped ZIP codes for manual inspection
print("Unmapped ZIP codes:")
print(unmapped_zips['committee_zip'].unique())

Unmapped ZIP codes:
['98697' '38102' '9206-3601' '72883' '98205' '98149' '90836' '99288'
 '98285' '98018' '98301' '98924-8644']


In [ ]:
df_new_clean1_copy[df_new_clean1_copy['committee_county'].isnull()][[ 'committee_acronym','committee_address', 'committee_city', 'committee_county',
                    'committee_state', 'committee_zip', 'committee_email',
                     'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
416,Citizens For Yelm,PO BOX 1537,Yelm,None,WA,98697,citizens4yelmschools@gmail.com,NaN,14222.36
1635,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
1636,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
1637,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
1638,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
1640,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
1641,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
2392,NaN,"9116 E. Sprague, #118",Spokane Valley,None,WA,9206-3601,matt@mchawkins.org,matt@HnA-Cap.com,14015.71
2865,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00
2866,NaN,942 S Shady Grove Road,Memphis,None,TN,38102,Fedexpac@fedex.com,NaN,0.00


In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd

# Initialize the geolocator
geolocator = Nominatim(user_agent="county_mapper")

# Function to get the county name using city and state
def get_county_from_city_state(row):
    try:
        if pd.isnull(row['committee_county']) and pd.notnull(row['committee_city']) and pd.notnull(row['committee_state']):
            location = geolocator.geocode(f"{row['committee_city']}, {row['committee_state']}, USA")
            if location and location.raw.get('address'):
                return location.raw['address'].get('county')
        return row['committee_county']  # Return existing county if not null
    except Exception as e:
        return row['committee_county']  # If any issue occurs, leave it as is

# Apply the function to fill missing committee_county using city and state
df_new_clean1_copy['committee_county'] = df_new_clean1_copy.apply(get_county_from_city_state, axis=1)

# Check rows that still have missing counties
unmapped_rows = df_new_clean1_copy[df_new_clean1_copy['committee_county'].isnull()]

# Log unmapped rows for manual inspection
print("Remaining unmapped rows:")


Remaining unmapped rows:
                    id    filer_id filer_type              registered  \
416      co-2023-37024  CITISY 597         CO                     NaN   
1635     co-2013-35568  FEDEE  120         OS                     NaN   
1636     co-2014-35568  FEDEE  120         OS                     NaN   
1637     co-2015-35568  FEDEE  120         OS                     NaN   
1638     co-2016-35568  FEDEE  120         OS                     NaN   
1640     co-2017-35568  FEDEE  120         OS                     NaN   
1641     co-2023-35568  FEDEE  120         OS                     NaN   
2392   ca-2024-3305493  HAWKM--987         CA  05/08/2024 12:00:00 AM   
2865     co-2018-35568  FEDEE  120         OS                     NaN   
2866     co-2019-35568  FEDEE  120         OS                     NaN   
2867     co-2020-35568  FEDEE  120         OS                     NaN   
3040     co-2011-10288  KENTFF 035         CO                     NaN   
3738      co-2007-3841  CO

In [ ]:
df_new_clean2_main[['committee_acronym','committee_address', 'committee_city', 'committee_county',
                    'committee_state', 'committee_zip', 'committee_email',
                     'candidate_email','contributions_amount']]

,committee_acronym,committee_address,committee_city,committee_county,committee_state,committee_zip,committee_email,candidate_email,contributions_amount
0,NaN,10 SABLE COURT,SEQUIM,CLALLAM,WA,98382,KEVINVANDEWEGE@HOTMAIL.COM,kevinvandewege@hotmail.com,32202.89
1,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,Clark,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,981.62
2,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,Clark,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,32805.76
4,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,Clark,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,0.00
13,NaN,"620 SE 168th Avenue, N-55",VANCOUVER,Clark,WA,98684-8428,LARRYDEV1@OUTLOOK.COM,NaN,10130.00
...,...,...,...,...,...,...,...,...,...
52259,NaN,"5426 N. Rd. 68, Ste D Box 220",Pasco,WALLA WALLA,WA,99301,nikkitorresforstatesenate@gmail.com,nikkitorresforstatesenate@gmail.com,14100.00
52260,NaN,PO Box 27113,Seattle,KING,WA,98165,fredforport@gmail.com,NaN,1.07
52261,NaN,2703 3rd street,Union Gap,YAKIMA,WA,98903,edperezjr84@gmail.com,edperezjr84@gmail.com,1993.49
52263,NaN,PO Box 37,North Bonneville,SKAMANIA,WA,98639,voteforest@gmail.com,voteforforest@gmail.com,5188.37


In [ ]:
df_new_clean2_main.committee_county.unique()

array(['CLALLAM', 'Clark', 'Walla Walla', 'King', 'KING', 'THURSTON',
       'Thurston', 'Spokane', 'District of Columbia', 'Various',
       'PACIFIC', 'KITSAP', 'COWLITZ', 'Whatcom', 'BENTON', 'WHITMAN',
       'YAKIMA', 'Snohomish', 'Pierce', 'WHATCOM', 'Benton', 'Cowlitz',
       'Chelan', 'STEVENS', 'MASON', 'PIERCE', 'ASOTIN', 'SNOHOMISH',
       'CLARK', 'OKANOGAN', 'LEWIS', 'Mason', 'Yakima', 'Jefferson',
       'KITTITAS', 'Columbia', 'GRANT', 'Kittitas', 'Clackamas',
       'Grays Harbor', 'Clallam', 'SPOKANE', 'ISLAND', 'Okanogan',
       'CHELAN', 'WALLA WALLA', 'San Juan', 'FERRY', 'JEFFERSON',
       'Douglas', 'Grant', 'SKAGIT', 'Skagit', 'Lewis', "Prince George's",
       'GRAYS HARBOR', 'Kings', 'Hennepin', 'Franklin', 'New York',
       'Wahkiakum', 'KLICKITAT', 'Clatsop', 'Kitsap', 'Montgomery',
       'Arlington', 'WAHKIAKUM', 'Whitman', 'SAN JUAN', 'PEND OREILLE',
       'Multnomah', 'DOUGLAS', 'ADAMS', 'GARFIELD', 'LINCOLN', 'FRANKLIN',
       'Island', 'Blue Eart

In [ ]:
def main():
    """
    Main function to process and analyze campaign finance data
    """
    # data_path = '/content/drive/MyDrive/VAD_Final_Project/'
    # Read the data
    df_new = df_new_clean2_main

    # Clean the data
    cleaned_data = clean_campaign_finance_data(df_new)

    # Create analysis views
    analysis_views = create_dashboard_views(cleaned_data)

    # Save cleaned data
    cleaned_data.to_csv('cleaned_campaign_finance_data.csv', index=False)

    # Save analysis views
    for view_name, view_data in analysis_views.items():
        view_data.to_csv(f'{view_name}.csv', index=False)

    # Print summary of processing
    print("Data processing completed successfully!")
    print(f"Total records processed: {len(cleaned_data)}")
    print("\nUnique counties found:", cleaned_data['committee_county'].nunique())
    print("\nRecords by region:")
    print(cleaned_data['region'].value_counts())

if __name__ == "__main__":
    main()

<ipython-input-4-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-4-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-4-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
<ipython-input-4-4416e1b4735a>:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To 

Data processing completed successfully!
Total records processed: 10148

Unique counties found: 40

Records by region:
region
Puget Sound    5134
Western WA     1596
Central WA     1235
Coastal        1087
Eastern WA      835
Other           261
Name: count, dtype: int64


In [ ]:
cleaned_df = pd.read_csv('cleaned_campaign_finance_data.csv')

<ipython-input-107-1ec80384d649>:1: DtypeWarning: Columns (32,33,45,48,49,50,51,52,74) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_df = pd.read_csv('cleaned_campaign_finance_data.csv')


In [ ]:
cleaned_df.committee_county.unique()

array(['CLALLAM', 'CLARK', 'WALLA WALLA', 'KING', 'THURSTON', 'SPOKANE',
       'DISTRICT OF COLUMBIA', 'VARIOUS', 'PACIFIC', 'KITSAP', 'COWLITZ',
       'WHATCOM', 'BENTON', 'WHITMAN', 'YAKIMA', 'SNOHOMISH', 'PIERCE',
       'CHELAN', 'STEVENS', 'MASON', 'ASOTIN', 'OKANOGAN', 'LEWIS',
       'JEFFERSON', 'KITTITAS', 'COLUMBIA', 'GRANT', 'CLACKAMAS',
       'GRAYS HARBOR', 'ISLAND', 'SAN JUAN', 'FERRY', 'DOUGLAS', 'SKAGIT',
       'PRINCE GEORGE', 'KINGS', 'HENNEPIN', 'FRANKLIN', 'NEW YORK',
       'WAHKIAKUM', 'KLICKITAT', 'CLATSOP', 'MONTGOMERY', 'ARLINGTON',
       'PEND OREILLE', 'MULTNOMAH', 'ADAMS', 'GARFIELD', 'LINCOLN',
       'BLUE EARTH', 'LOS ANGELES', 'ALAMEDA', 'DALLAS', 'SKAMANIA',
       'HARRIS', 'SAN MATEO', 'COLLIN', 'WASHINGTON', 'SEDGWICK',
       'HUDSON', 'CHITTENDEN', 'CUYAHOGA', 'LAKE', 'ST. LOUIS', 'MCLEAN',
       'MARIN', 'FULTON', 'SUFFOLK', 'FAIRFAX (CITY)', 'SACRAMENTO',
       'ALEXANDRIA'], dtype=object)

In [ ]:
for i in cleaned_df.isnull().sum():
  print(i)

0
0
0
10938
17577
21051
21034
0
337
0
0
0
18485
1
1
0
4516
0
1235
11154
5046
10937
10937
10540
10549
10908
10542
12874
10944
10937
0
0
18594
18300
17495
13507
0
8943
4246
21056
21056
21056
11956
13789
0
18594
0
10455
20621
20231
20158
20126
19464
471
471
4990
471
6616
0
0
0
0
0
0
0
0
263
0
872
10937
0
18583
337
0
10119
0
0
0
0
0
337
337
337
2916
